# TPI Teoría de Control: Simulación de Controlador PD

**Alumno:** Matías Ezequiel Nuñez
**Materia:** Teoría de Control (K4572)

Este notebook implementa la simulación dinámica del sistema de **Rate Limiting** modelado como un lazo de control cerrado.
A diferencia del enfoque clásico de Token Bucket (PI), aquí implementamos:

1.  **Controlador PD:** $G_c(s) = K_p + K_d \cdot s$.
2.  **Actuador con Memoria:** El mecanismo de asignación de recursos (Bucket/Autoscaler) actúa como un integrador puro en el lazo directo.
3.  **Realimentación Unitaria:** $H(s) = 1$.

El objetivo es validar que el sistema es estable y presenta error estacionario nulo ($e_{ss}=0$) gracias a la naturaleza "Tipo 1" del lazo completo, a pesar de que el controlador es PD.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, RadioButtons

# Configuración de Matplotlib para interactividad
# Nota: Si usas Jupyter Lab o VSCode, instala 'ipympl' y usa: %matplotlib widget
# Si usas Jupyter Notebook clásico, usa: %matplotlib notebook
%matplotlib widget
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
class SimuladorRateLimiter:
    def __init__(self):
        self.dt = 0.05  # Paso de tiempo
        self.sim_time = 30.0 # Duración
        self.t = np.arange(0, self.sim_time, self.dt)
        self.n = len(self.t)
        
        # Valores iniciales del controlador (Default)
        self.Kp = 0.5
        self.Kd = 0.2
        
        # Escenario actual
        self.escenario = 'Rafagas' # 'Rafagas' o 'DoS'

    def generar_escenario(self):
        """Genera las señales de Referencia (Setpoint) y Perturbación (D)"""
        # Referencia (R): Nivel de servicio deseado (ej. 100 req/s)
        R = np.ones(self.n) * 100.0 
        
        # Perturbación (D): Tráfico entrante NO deseado o carga extra
        D = np.zeros(self.n)
        
        if self.escenario == 'Rafagas':
            # Ráfagas cortas en t=5s y t=15s
            D[int(5/self.dt):int(7/self.dt)] = 150.0  # Pico fuerte
            D[int(15/self.dt):int(18/self.dt)] = 80.0 # Pico medio
            
        elif self.escenario == 'DoS':
            # Ataque sostenido desde t=5s hasta el final
            D[int(5/self.dt):] = 400.0 # Carga masiva constante
            
        return R, D

    def ejecutar_simulacion(self):
        R, D = self.generar_escenario()
        
        # Inicialización de vectores
        Y = np.zeros(self.n) # Salida (Throughput real)
        e = np.zeros(self.n) # Error
        u = np.zeros(self.n) # Señal de control
        
        # Variables de estado
        capacidad_actual = 0.0 # Variable interna del actuador (Memoria/Bucket)
        y_prev = 0.0
        e_prev = 0.0
        
        # Bucle de simulación
        for i in range(1, self.n):
            # 1. Medición y Cálculo del Error (H(s)=1)
            e[i] = R[i] - y_prev
            
            # 2. Controlador PD: u(t) = Kp*e(t) + Kd*de(t)/dt
            derivativa = (e[i] - e_prev) / self.dt
            u[i] = (self.Kp * e[i]) + (self.Kd * derivativa)
            
            # 3. Actuador con "Memoria" (Efecto Integrador)
            # Acumula la señal de control u(t). Esto convierte al sistema en Tipo 1.
            capacidad_actual += u[i] * self.dt * 5.0 # Factor de ganancia
            
            # Limitaciones físicas (No hay capacidad negativa)
            if capacidad_actual < 0: capacidad_actual = 0
            
            # 4. Planta (Lag de primer orden) + Perturbación
            tau = 0.5 
            entrada_planta = capacidad_actual - D[i] * 0.2 
            Y[i] = (self.dt * entrada_planta + tau * y_prev) / (tau + self.dt)
            
            # Actualizar previos
            y_prev = Y[i]
            e_prev = e[i]
            
        return self.t, R, Y, u, e, D

## Laboratorio Interactivo

Utilice los controles a continuación para modificar los parámetros del controlador en tiempo real:

* **Sliders Kp / Kd:** Ajuste las ganancias Proporcional y Derivativa. Observe cómo un $K_d$ mayor amortigua las oscilaciones.
* **Radio Buttons:** Cambie entre escenarios:
    * *Ráfagas:* Evalúa la respuesta transitoria.
    * *Ataque DoS:* Evalúa la estabilidad y el error en estado estacionario.

In [ ]:
sim = SimuladorRateLimiter()

# Crear figura y ejes
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(9, 8), sharex=True)
plt.subplots_adjust(left=0.1, bottom=0.25, hspace=0.4)

# Configuración de gráficas iniciales
l_ref, = ax1.plot([], [], 'k--', label=r'$\theta_i$ (Setpoint)', linewidth=1.5)
l_y, = ax1.plot([], [], 'b-', label='Salida (Y)', linewidth=2)
l_pert, = ax1.plot([], [], 'r:', label='Perturbación (D)', alpha=0.6)

l_err, = ax2.plot([], [], 'g-', label='Error (e)', linewidth=1.5)
l_u, = ax3.plot([], [], 'm-', label='Señal de Control (u)', linewidth=1.5)

# Etiquetas
ax1.set_title('Respuesta Temporal del Sistema')
ax1.set_ylabel('Throughput (req/s)')
ax1.legend(loc='upper right')
ax1.grid(True, linestyle=':', alpha=0.6)

ax2.set_ylabel('Error $e(t)$')
ax2.grid(True, linestyle=':', alpha=0.6)

ax3.set_ylabel('Control $u(t)$')
ax3.set_xlabel('Tiempo (s)')
ax3.grid(True, linestyle=':', alpha=0.6)

# Función de actualización
def update(val):
    sim.Kp = s_kp.val
    sim.Kd = s_kd.val
    t, R, Y, u, e, D = sim.ejecutar_simulacion()
    
    l_ref.set_data(t, R)
    l_y.set_data(t, Y)
    l_pert.set_data(t, D)
    l_err.set_data(t, e)
    l_u.set_data(t, u)
    
    ax1.relim(); ax1.autoscale_view()
    ax2.relim(); ax2.autoscale_view()
    ax3.relim(); ax3.autoscale_view()
    fig.canvas.draw_idle()

# Sliders
ax_kp = plt.axes([0.15, 0.1, 0.65, 0.03], facecolor='lightgoldenrodyellow')
ax_kd = plt.axes([0.15, 0.05, 0.65, 0.03], facecolor='lightgoldenrodyellow')
s_kp = Slider(ax_kp, 'Kp (Prop.)', 0.0, 2.0, valinit=0.5, valstep=0.05)
s_kd = Slider(ax_kd, 'Kd (Deriv.)', 0.0, 2.0, valinit=0.2, valstep=0.05)

s_kp.on_changed(update)
s_kd.on_changed(update)

# Botones
rax = plt.axes([0.82, 0.05, 0.15, 0.15], facecolor='#f0f0f0')
radio = RadioButtons(rax, ('Rafagas', 'DoS'))

def change_scenario(label):
    sim.escenario = label
    update(None)
radio.on_clicked(change_scenario)

# Inicializar
update(None)
plt.show()

### Conclusiones de la Simulación

**1. Escenario Ráfagas (Transitorio):**
Se observa cómo el sistema reacciona ante picos de tráfico.
- Al aumentar **Kp**, el sistema reacciona más rápido pero puede presentar sobrepicos (overshoot).
- Al aumentar **Kd**, se introduce amortiguamiento, suavizando la respuesta y reduciendo oscilaciones, validando la acción derivativa del controlador PD.

**2. Escenario Ataque DoS (Estado Estacionario):**
Ante una perturbación sostenida (escalón):
- La salida $Y(t)$ regresa al valor de referencia (Setpoint).
- El error $e(t)$ converge a **CERO**.
- **Justificación Teórica:** Aunque el controlador es PD (sin término integral explícito), el actuador posee memoria (acumulación de recursos/tokens). Esto añade un polo en el origen al lazo abierto, convirtiendo al sistema en **Tipo 1**, lo que garantiza $e_{ss}=0$ ante entradas escalón.